In [1]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoConfig, BitsAndBytesConfig, GenerationConfig
from peft import LoraConfig, PeftModel
from datasets import Dataset, load_dataset


In [2]:
torch.cuda.get_device_name()

'NVIDIA RTX A5000 Laptop GPU'

In [3]:
# Quantization configuration
# we are going to change the model from fp16 to int8
bnb_config = BitsAndBytesConfig(
    load_in_8bit = True,
    
)

In [6]:
# load model with automodel and autotokenizer
model_id = "meta-llama/Meta-Llama-3-8B"
model = AutoModelForCausalLM.from_pretrained(model_id,
                                            quantization_config = bnb_config,
                                            device_map = "cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
device = 'cuda'

#ext_no_template = '<s>[INST]Hello, where are you from?[\INST]<\s>'
text=[
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

#okenized_text_no_template  = tokenizer(text_no_template)
#tokenized_text = tokenizer.apply_chat_template(text, tokenize = True, return_tensors= 'pt')

tokenized_text = tokenizer("I like this LLM what should i do, ",  return_tensors = 'pt')

#rint("Tokenized with no template")
#rint(tokenized_text_no_template)
#rint('Tokenized with apply template function')
print(tokenized_text)

{'input_ids': tensor([[   40,  1093,   420,   445, 11237,  1148,  1288,   602,   656,    11,
           220]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [13]:
#model.to(device)
model_input = tokenized_text.to(device)

In [15]:
generated_id = model.generate(model_input['input_ids'], max_new_tokens = 20, do_sample=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
C:\Users\tchen\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\llama\modeling_llama.py:670: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [16]:
decoded = tokenizer.batch_decode(generated_id)
print(decoded)

['I like this LLM what should i do, 24 hour essay writer LLM in Banking and Finance Law at QMUL or LLM in Corporate']


In [22]:
del model
import gc
gc.collect()
torch.cuda.empty_cache()

NameError: name 'model' is not defined

In [31]:
# load model using pipeline
pipe = pipeline("text-generation",
                model = 'meta-llama/Llama-2-7b-chat-hf',
                device_map = 'auto'
                )

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [21]:
text_gen = pipe("I really like apple so " , truncation = True ,max_length = 50)
print(text_gen)

[{'generated_text': 'I really like apple so 1000 apple is really good.\nI have a question about the question.\nWhat if the apple is 1000 apple?\nIt’s 1000 apple.\nI'}]


In [30]:
#del pipe
torch.cuda.empty_cache()
gc.collect()
gc.collect()

0